In [1]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
%matplotlib inline

/scratch/abdu/833135/ipykernel_31631/1028975186.py:8: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [2]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [3]:
root = 'abdu/CSRNet-pytorch/IDCIA'

In [4]:
#now generate the ShanghaiA's ground truth
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_A_test]

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
import json

with open('test.json', 'r') as f:
  data = json.load(f)


In [5]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [6]:
model = CSRNet()

In [7]:
model = model.to(device)

In [9]:
checkpoint = torch.load('0model_best.pth.tar')

In [10]:
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [34]:
mae = 0
transform=transforms.Compose([transforms.ToTensor(),
                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])
for i in range(len(data)):
    img = Image.open(data[i]).convert('RGB')
    img = transform(img).cuda()
    # img[0,:,:]=img[0,:,:]-92.8207477031
    # img[1,:,:]=img[1,:,:]-95.2757037428
    # img[2,:,:]=img[2,:,:]-104.877445883
    img = img.to(device)
    #img = transform(Image.open(img_paths[i]).convert('RGB')).cuda()
    gt_file = h5py.File(data[i].replace('.tiff','.h5').replace('images','ground_truth'),'r')
    #print(data[i])
    groundtruth = np.asarray(gt_file['density'])
    output = model(img.unsqueeze(0))
    #print(f"The prediction is: {int(output.detach().cpu().sum().numpy())} and the ground truth is {np.sum(groundtruth)}")
    mae += abs(output.detach().cpu().sum().numpy()-np.sum(groundtruth))
    print (i,mae)
    print(len(data))
print (mae/len(data))

0 2.2181310653686523
52
1 3.6599674224853516
52
2 4.589557647705078
52
3 8.9922513961792
52
4 25.660548210144043
52
5 35.01821804046631
52
6 50.212714195251465
52
7 56.964669704437256
52
8 60.96007299423218
52
9 72.69265127182007
52
10 75.42641925811768
52
11 82.02518939971924
52
12 88.27704238891602
52
13 92.31480026245117
52
14 96.3329119682312
52
15 105.10876417160034
52
16 105.26580786705017
52
17 112.44543814659119
52
18 112.47018933296204
52
19 118.19524312019348
52
20 124.07465863227844
52
21 124.88626980781555
52
22 133.48138642311096
52
23 136.7087299823761
52
24 143.2483561038971
52
25 148.81314539909363
52
26 158.40725779533386
52
27 160.66257452964783
52
28 173.80035853385925
52
29 179.68850016593933
52
30 191.4583899974823
52
31 196.7242820262909
52
32 199.202867269516
52
33 202.50807762145996
52
34 209.14150285720825
52
35 209.5620675086975
52
36 209.589204384014
52
37 216.65027864091098
52
38 222.52914435975254
52
39 227.02528007142246
52
40 231.22211940400302
52
41 255.